In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/new data test

/content/gdrive/MyDrive/new data test


In [3]:
!pip install gdown

In [4]:
!pip install kaggle


In [5]:
!pip install opendatasets
!pip install pandas

In [6]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 10.6 MB/s eta 

In [7]:
# !git clone https://github.com/facebookresearch/dinov2.git

In [8]:
# %cd /dinov2
# !pip install -r requirements.txt
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00


In [9]:
# %cd /dinov2

In [10]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import faiss
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
import warnings

warnings.filterwarnings("ignore")
import gradio as gr

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the DINO model
dino = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")
dino.to(device)
dino.eval()

# Define the image transformations
image_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),  # Convert to RGB format
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# Function to extract features from an image
def extract_features(image):
    image_tensor = image_transforms(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = dino(image_tensor).float()
    return features


# Function to perform nearest neighbor search
def nearest_neighbor_search(query_features, dataset_features, k=10):
    index = faiss.IndexFlatL2(query_features.shape[1])
    dataset_features = torch.cat(dataset_features, dim=0).cpu().numpy()
    index.add(dataset_features)
    distances, indices = index.search(query_features.cpu().numpy(), k)
    return indices


# Function to return the nearest neighbor images
def return_nearest_neighbor_images(query_image):
    query_features = extract_features(query_image)
    indices = nearest_neighbor_search(query_features, dataset_features)
    nearest_neighbor_image_paths = []
    for i in range(k):
        image_path = os.path.join(dataset_path, dataset_images[indices[0][i]])
        nearest_neighbor_image_paths.append(image_path)
    return nearest_neighbor_image_paths


# Gradio interface
dataset_path = "/content/gdrive/MyDrive/new data test"
dataset_images = os.listdir(dataset_path)
dataset_features = []
for filename in dataset_images:
    image_path = os.path.join(dataset_path, filename)
    if (
        filename.endswith(".png")
        or filename.endswith(".jpg")
        or filename.endswith(".jpeg")
    ):
        image = Image.open(image_path)
        features = extract_features(image)
        dataset_features.append(features)

k = 5  # Number of nearest neighbors to retrieve

inputs = gr.inputs.Image(type="pil", label="Input Image")
# outputs = gr.outputs.Image(type="pil", label="Returned Images", multiple=True)

outputs = gr.Gallery()

demo = gr.Interface(
    fn=return_nearest_neighbor_images,
    inputs=inputs,
    outputs=outputs,
    title="Recommendation fashion",
    description="Please, uploading an image to  get the recommendation images.",
)

demo.launch()

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:03<00:00, 106MB/s]


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [14]:
pip install streamlit


In [15]:
pip install tensorflow

In [16]:
pip install keras


In [17]:
import streamlit as st
import os
from PIL import Image
import numpy as np
import pickle
import tensorflow
from tensorflow.keras.preprocessing import image
uploaded_file = st.file_uploader("Choose an image")
if uploaded_file is not None:
    if save_uploaded_file(uploaded_file):
        # display the file
        display_image = Image.open(uploaded_file)
        st.image(display_image)
        # feature extract
        features = feature_extraction(os.path.join("uploads",uploaded_file.name),model)
        #st.text(features)
        # recommendention
        indices = recommend(features,feature_list)
        # show
        col1,col2,col3,col4,col5 = st.beta_columns(5)

        with col1:
            st.image(filenames[indices[0][0]])
        with col2:
            st.image(filenames[indices[0][1]])
        with col3:
            st.image(filenames[indices[0][2]])
        with col4:
            st.image(filenames[indices[0][3]])
        with col5:
            st.image(filenames[indices[0][4]])
    else:
        st.header("Some error occured in file upload")

2023-09-27 06:47:05.434 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [19]:
!streamlit run https://raw.githubusercontent.com/streamlit/demo-uber-nyc-pickups/master/streamlit_app.py





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.218.221:8501

  Stopping...
^C
